# "Regular" click2click co-visitation matrix
This notebook prepares one of click-to-click co-visitation matrixes. It is used to generate candidates and calculate features for the clicks model.
There are two different outputs, both for the cross-validation and for test datasets. One of those outputs is a full co-visitation matrix, and it is used to generate features for the clicks model. Another one is the same matrix reduced to top_70 results, and it is used for candidate generation.
## Imports and definitions

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
        
import gc
from humanize import naturalsize

# functions and classes common for several notebooks of current project
import otto_common

In [2]:
class CalculateCovisitationMatrixRegular(otto_common.CalculateCovisitationMatrix):
    '''
    Child class with logic specific to this type of co-visitation matrix.
    It counts all aids before or after each aid if time between those events was less than 5 minutes.
    Weight value is calculated in a way that makes later events more important.
    '''

    def __init__(self, *args, timebase, **kwargs):
        self.timebase = timebase
        super().__init__(*args, **kwargs)

    def calculate_weights(self, df_i):
        df_i = df_i.sort_values(['session','ts'],ascending=[True,False])
        df_i = df_i.reset_index(drop=True)
        df_i = df_i.merge(df_i,on='session')
        df_i = df_i.loc[ ((df_i.ts_x - df_i.ts_y).abs()< 5 * 60) & (df_i.aid_x != df_i.aid_y) ]
        df_i = df_i[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y']) 
        df_i['wgt'] = (1 + 3*(df_i.ts_x - self.timebase)/3024000)   
        df_i = df_i[['aid_x','aid_y','wgt']]
        df_i = df_i.loc[df_i['aid_x'].isin(self.aid_list)]
        df_i.wgt = df_i.wgt.astype('float32')
        df_i = self.groupby_reset_and_reduce(df_i)
        return df_i

## Co-visitation matrix for cross-validation dataset

In [3]:
# Make a list of AIDs, that appear in cross-validation sets.
# aid_x is filtered to only include aids from the list, while aid_y are not filtered
# This reduces size of resulting matrix.
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/cv_inputs.parquet'
trunked_sessions_path2 = '/kaggle/input/otto-prepare-cv/cv_inputs2.parquet'
cv_list = otto_common.build_aid_list(trunked_sessions_path, trunked_sessions_path2)

In [4]:
# Input parameters and matrix calculation itself.
click_data_cv = '/kaggle/input/otto-prepare-cv/cv_train.parquet'
n_splits = 200
n_splits_concat = 25
timebase_cv = 1658700000

covisitation_cv = CalculateCovisitationMatrixRegular(n_splits, n_splits_concat, cv_list, timebase=timebase_cv)
df_matrix_cv = covisitation_cv.generate_covisitation_matrix(click_data_cv)
df_matrix_cv.to_parquet('regular_click2click_matrix_cv.parquet')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [5]:
# Reduce the matrix to top_n aid_y for each aid_x
top_n = 70
df_matrix_cv = covisitation_cv.get_top_n(df_matrix_cv, top_n)
# Check dataframe size
size = df_matrix_cv.memory_usage(deep='True').sum()
print(naturalsize(size))
# Save output.
df_matrix_cv.to_parquet('regular_click2click_matrix_cv_top_n.parquet')

981.1 MB


In [6]:
del df_matrix_cv, cv_list, covisitation_cv
gc.collect()

21

## Co-visitation matrix for test dataset
Exactly the same steps to prepare co-visitation matrix for the test dataset.

In [7]:
trunked_sessions_path = '/kaggle/input/otto-prepare-cv/test.parquet'
full_aid_list = otto_common.build_aid_list(trunked_sessions_path)

In [8]:
click_data_full = '/kaggle/input/otto-prepare-cv/train_full.parquet'
n_splits = 350
n_splits_concat = 40
top_n = 70
timebase_test = 1659304800

covisitation_test = CalculateCovisitationMatrixRegular(n_splits, n_splits_concat, full_aid_list, timebase=timebase_test)
df_matrix_full = covisitation_test.generate_covisitation_matrix(click_data_full)
df_matrix_full.to_parquet('regular_click2click_matrix_test.parquet')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
df_matrix_full = covisitation_test.get_top_n(df_matrix_full, top_n)

# Check dataframe size
size = df_matrix_full.memory_usage(deep='True').sum()
print(naturalsize(size))

df_matrix_full.to_parquet('regular_click2click_matrix_test_top_n.parquet')

916.9 MB
